## Install packages

In [1]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
# from transformers import get_linear_schedule_with_warmup
# from transformers.optimization import AdamW
import sys
from sys import platform
import re
from pathlib import Path

import matplotlib.pyplot as plt
%matplotlib inline

## Set up GPU

In [2]:
if torch.cuda.is_available():
  device = torch.device("cuda")
  print(f'There are {torch.cuda.device_count()} GPU(s) available.')
  print('Device name:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


## Load Dataset

In [3]:
data_path = Path('data')

In [4]:
train_df = pd.read_csv(data_path / 'train.csv')
test_df = pd.read_csv(data_path / 'test.csv')
val_df = pd.read_csv(data_path / 'val.csv')

print(f"train: {len(train_df)}")
print(f"test: {len(test_df)}")
print(f"val: {len(val_df)}")

train: 67349
test: 1821
val: 872


# Fine-Tuned Bert Classifier

## PyTorch DataLoader

In [5]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from datasets import SST2Dataset, DatasetType
from transformers import BertTokenizer

# Convert other data types to torch.Tensor
# train_labels = torch.tensor(y_train)
# val_labels = torch.tensor(y_val)

# For fine-tuning BERT, the authors recommend a batch size of 16 or 32.
batch_size = 32
MAX_LEN = 64

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Create the DataLoader for our training set
# train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_data = SST2Dataset(DatasetType.TRAIN, tokenizer, max_seq_len = MAX_LEN)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set
# val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_data = SST2Dataset(DatasetType.VAL, tokenizer, max_seq_len = MAX_LEN)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

ModuleNotFoundError: No module named 'datasets'

## Train

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup
from BertClassifier import BertClassifier

def initialize_model(epochs=4):
    """Initialize the Bert Classifier, the optimizer and the learning rate scheduler.
    """
    # Instantiate Bert Classifier
    bert_classifier = BertClassifier(freeze_bert=True)

    # Tell PyTorch to run the model on GPU
    bert_classifier.to(device)

    # Create the optimizer
    optimizer = AdamW(bert_classifier.parameters(),
                      lr=5e-5,    # Default learning rate
                      eps=1e-8    # Default epsilon value
                      )

    # Total number of training steps
    total_steps = len(train_dataloader) * epochs

    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler

In [ ]:
%%time

import utils

utils.set_seed(42)    # Set seed for reproducibility
bert_classifier, optimizer, scheduler = initialize_model(epochs=2)

utils.train(bert_classifier, train_dataloader, val_dataloader, epochs=2, evaluation=True)

## Evaluate

In [ ]:
from BertClassifier import bert_predict
from utils import evaluate_roc

# Compute predicted probabilities on the test set
probs = bert_predict(bert_classifier, val_dataloader, device)

# Evaluate the Bert classifier
evaluate_roc(probs, y_val)

# Influence Function

In [ ]:
# test